![figure](lab6/politecnico_h-01.png)

# **Programmable Electronics**
#### Bachelor's Degree in Electrical and Computer Engineering

## Laboratory Work 6 - Serial Communication Systems - UART and PS2 

-----------------------


## Introduction  ##
In this work we will implement and test two communication interfaces: a) UART, which establishes an asynchronous serial communication system; and b) PS2, which establishes a synchronous serial communication system. This document does not provide details on how to use Vivado. Whenever you have questions at this level, consult the Lab1 statement.


### Objectives ###

In this work we propose the implementation of three communication systems, which are implemented using state machines with datapath:
* Bidirectional UART Communication System.
* PS2 Communication System for keyboards.
* PS2 Communication System for mice.


In the instructions below  **{sources}** refers to `C:\Xilinx\EEC_EP\sources` and **{labs}** refers to `(C:\Xilinx\EEC_EP\labs)`


_______________________

## Step 1 - Bidirectional UART ##

### Step 1.1  ### 

The block diagram of a bidirectional UART is represented in the figure below. It consists of a shared timer ( baud rate generator ), a transmitter, a receiver and two FIFO memories serving as interface modules. The VHDL modules for each of these components are provided in the files: **mod_m_counter.vhd**, **uart_tx.vhd**, **uart_rx.vhd** and **fifo.vhd**. Note that the last module is the same as that used in HandsOn 3. The structural VHDL description of the block diagram is provided in the file **uart.vhd**. 
 

<div>
<img src="lab6/fig1.png" width=700/>
</div>
                                   

* Start by downloading the provided source files (Github or Moodle) and saving them in `{sources}/lab6`.


* Open Vivado from the icon on your desktop or: `Start > All Programs > Xilinx Design Tools > Vivado 2022.1`. 


* Select **Create Project** to start the wizard. In the *Create A New Vivado Project* dialog box , select **Next**.


* Click the Browse button in the project location field of the new project form, navigate to `c:/Xilinx/EEC_EP/labs` and click **Select**. Enter *lab5* in the project name field. Make sure the *Create Project Subdirectory* box is checked. Click **Next**.


* Select the *RTL Project* option in the*Project Type* form and click **Next**. Make sure the *Do not specify sources at this time* box is checked.


* In the *Default Part* form , select the *Boards* option and choose the Basys3 board. Click **Next** and then **Finish** to create the Vivado project.


* In the *Sources* panel, select *Add Sources* and follow the necessary steps (*Add or create design sources*) to add all the necessary VHDL files (described above).


* Open the **uart.vhd** file in Vivado. Look at the code and verify that it describes the block diagram shown above. Note that the generic DVSR ( baud rate divisor ) should have a value of 326 if the circuit is implemented on the board (which has a 100MHz clock) but that it can be smaller (DVSR=2) for simulation.


* Add the simulation stimuli (*testbench*) file **uart_tb** to the project. Observe the processes that generate the input stimuli.


* Run the simulation (with the generic **DVSR=2**) and analyze the system's operation.


* Close the simulation view and reset the DVSR to its original value (**DVSR=326**).



### Step 1.2  ### 

To verify the operation of the UART in the FPGA we will use the Hyperterminal of a PC and a button on the development board, as shown in the figure below. In this circuit, the serial port (RS-232) of the development board is connected to the serial port of a PC. The characters sent from the PC must be received by the UART and stored in the FIFO of the receiver. The stored data is then changed (by an incrementing circuit) and retransmitted to the PC, through the UART. A pushbutton (with a debouncing circuit) is used to control the reading and writing of data in the FIFO. 
 

<div>
<img src="lab6/fig2.png" width=500/>
</div>
            
            
For example, when we write HAL in Hyperterminal and send it to the FPGA, the UART must receive the characters and store them in the receive FIFO. Then, if we press the pushbutton once, we will see the letter I being received in HyperTerminal (letter H +1). Pressing it twice more, we receive the remaining characters B and M.

* Add to the project the files describing the test module (**uart_test.vhd**) and the debounce circuit (**debounce.vhd**).


* Open the **uart_test.vhd** file and notice that the received data can be seen on a set of 8 LEDs and that the rx_full and rx_empty signals from the receive FIFO have been connected to two segments of one of the 7-segment displays (display 0).


* Synthesize and implement the module on the FPGA. Remember that you must first set this as the top module (Set as Top Module), and create an input/output pin file (based on the input and output signals of this entity).

* Turn on the Basys3 and program it. The USB cable you use to program the board will also serve as a serial communication cable, emulating an RS-232 connection.


* Open a Hyperterminal of your choice (I suggest **TeraTerm**) to interact with the development board, through the serial port. This program allows you to send characters in ASCII format. There are 128 characters defined in this format (alphabet, punctuation and control characters), which uses 7 bits to encode each character. If you feel the need, consult the ASCII character table at http://www.asciitable.com/ .


* Set Hyperterminal to  **19200baud, 8 bits de dados, 1 stop bit, zero parity bits, and no flow control**. If you use the TeraTerm application to communicate with the serial port, the procedure to follow is as follows.

  1.	In **New Connection**, choose serial communication and select the last detected COM port. Click **OK**. 

  2.	Go to **Setup -> Serial Port**, and configure the port acording to the specifications (Baud rate: 19200; Data: 8 bits; Parity: none; Stop: 1bit, Flow Control: none). Click **OK**. 

  3.	Go to **Setup -> Terminal**, and choose **Local Echo**. This allows you to observe the caracters sent in the terminal. Click **OK**.


* Now you are ready to communicate with the development board. Observe the UART operation on the LEDs and also on HyperTerminal. Type the HAL characters in HyperTerminal and then press the push button three times. See if you see what you expected on the LEDs and on HyperTerminal.

* Try with a few more characters and notice what happens when you try to transmit characters when the receiver's memory is empty (**rx_empty = '1'**) and what happens when it becomes full (**rx_full = '1'**). Test this after a memory reset and after filling it completely. 



______

## Step 2 - PS2 interface to a keyboard (PS2_RX) ##

### Step 2.1  ### 

The circuit shown in the figure below represents a PS2 receiver for keyboards, with extraction of the **make code** of the pressed key (let's consider that only one key is pressed at a time and that this key is a regular key, i.e., not extended) and storage in a reception FIFO.


<div>
<img src="lab6/fig3.png" width="700"/>
</div>


The PS2 data is received in the **ps2_rx.vhd** module and sent to a FIFO (the same one used in step 1 of this work, and has already been added to the project). This entire procedure is controlled by a state machine. In the **wait_brk** state, the FSM constantly monitors the received packets. When the **F0** (break code) packet is detected, it switches to the **get_code** state and waits for the next packet, which is already known to be the make code of the pressed key. Afterwards, the machine asserts the **code_done_tick = '1'** signal for one clock cycle and returns to the initial state (**wait_brk**).


* Add the files **ps2_rx.vhd** and **kb_code.vhd** (which instantiates the other modules and defines the FSM) to the project. Open this last file and observe the code that describes the FSM.


* Now add to the project the file **kb_test.vhd**, which describes the block diagram given below. This circuit allows to receive the make code of the pressed key, convert it to ASCII code and send it to the PC's Hyperterminal via UART. Open the files and see how this system is implemented.


<div>
<img src="lab6/fig4.png" width="700"/>
</div>


* Implement this circuit on the Basys 3. Remember that you need to set the **kb_test.vhd** module as the top module (*Set as Top*) and update the input/output pins file.

* Program the board, connect a keyboard to the USB port and check the operation of the PS2 keyboard through the PC's HyperTerminal.

* Try to answer the following questions:

  1. Why do I receive the * character in HyperTerminal when I press the SHIFT key? Where (in the code) is this behavior specified?

  2. Why does the code for a key only appear in HyperTerminal when it is released, and not when it is pressed?


______________

## Step 3 - PS2 interface to a mouse (PS2_RX + PS2_TX) ##

### Step 3.1 ### 

By combining the transmit and receive subsystems, we can form a bidirectional communication system, such as is required to use a PS2 mouse. The block diagram of this system is represented in the figure by the blue block. The **tx_idle** and **rx_en** signals serve to coordinate the transmit and receive operations. The code with the structural VHDL description of this circuit is provided in the file **ps2_rxtx.vhd**. The ps2_tx module is provided in the file **ps2_tx.vhd**.

<div>
<img src="lab6/fig5.png" width="500"/>
</div>

The complete mouse interface, provided in the **mouse.vhd** file, creates a new protocol layer in the communication system. The circuit has as outputs: **xm and ym**, which correspond to movements on the X and Y axes; **btnm**, which indicates the circuit status in 3 bits; and the **m_done_tick** signal, which is asserted when data is available.

The protocol layer is implemented as an FSMD, with seven states. The states **init1, init2 and init3** are executed after the system reboots, to put the mouse into stream mode. States **pack1, pack2 and pack3** are then used to receive the 3 data packets. In the final state, **done**, the m_done_tick signal is asserted. The machine then cycles through these last 4 states.


To test the circuit on the board, a circuit is provided that uses the mouse to control the LEDs on the development board. It should light up only 1 LED, which follows the movement of the mouse on the X axis. Pressing the right or left mouse button should light up either the left LED or the right LED.


* Add the file with the VHDL description of this circuit (**mouse_led.vhd**) to the project.

* Implement the circuit in Basys3. Remember that you must first define it as the top module and then create an input/output pins file.

* Program the board and check its operation, using a mouse connected to the Basys3's USB-A route.

* Analyze the code of the top module and answer the following questions.
 
 1. When I move the mouse on the y-axis the LEDs do not respond. Why?
 
 2. When you move the mouse along the y-axis, the LEDs light up slowly. If the mouse is supposed to increment its counters by 4 units every millimeter, why do they light up so slowly?
 
 3. In the **mouse.vhd** file there is a state machine with 7 states. Why are 7 states needed and what is the function of each one?



_________

## Challenge ##

Create a stimulus file that allows you to simulate the operation of the **kb_code.vhd** circuit. Simulate the arrival of data relating to a key press event **A** (1C F0 1C). Note that you will need to generate the ps2d and ps2c signals with the appropriate timings and with the correct values: 1) the clk clock frequency must be much higher than the ps2c clock frequency because of the filter; 2) the falling edge of ps2c must be halfway through the data in ps2d; 3) the signal in ps2d must start with a start bit, then the data (LSB first), parity and stop bit; 4) the ps2c and ps2d lines are normally '1'.

Run the simulation and check if the circuit works according to the flow diagram presented in the TP class slides.


